<a href="https://www.kaggle.com/code/lucascarpantonio/titanic-machine-learning-from-disaster?scriptVersionId=286172756" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/lucascarpantonio/titanic-machine-learning-from-disaster?scriptVersionId=285818002" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 1. Overview

In [ ]:
# loading training data (use local Kaggle input path)
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
train_data.info()
train_data.head(4)

In [ ]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
test_data.info()
test_data.head(4)

In [ ]:
train_X = train_data.drop(columns='Survived', errors='ignore')

train_y = train_data['Survived']


train_vs_test = pd.DataFrame({
	'train': pd.Series(train_X.columns.tolist()),
	'test': pd.Series(test_data.columns.tolist())
})

train_vs_test


## 3. Build Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

num = ['PassengerId','Age', 'Pclass','Fare', 'Parch'] #remove 'SibSp' to avoid redundancy
cat = ['Sex','Ticket','Embarked']

feature_cols = num + cat


X = train_data[feature_cols]
y = train_data['Survived']

In [ ]:

# 2) Trasformazioni
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num),
        ("cat", categorical_transformer, cat),
    ]
)

In [ ]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(train_X, train_y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=90,
    random_state=42
)

In [ ]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train_full, y_train)

# Preprocessing of validation data, get predictions
preds_valid = my_pipeline.predict(X_valid_full)


In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(
    my_pipeline, X_train_full, y_train,
    cv=5,
    scoring="accuracy"
)

scores.mean(), scores.std()

In [ ]:
from sklearn.metrics import mean_absolute_error


mae = mean_absolute_error(y_valid, preds_valid)
print("MAE:", mae)

### 4. Prediction

In [ ]:
my_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
])

my_pipeline.fit(X[num + cat], y)

preds = my_pipeline.predict(test_data[num + cat])

### 5. Conclusions

In [ ]:
output = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': preds
})
output.to_csv('submission.csv', index=False)
